# Grabbing Rotman Publication Data from Pubmed


## Overview

...

## Notes

    
    
[publications infographic here](http://www.immpressmagazine.com/109/)


I like the picture on [this page](http://www.mydesignshop.com/infographic-design)

Bokeh [annular wedge demo](http://bokeh.pydata.org/en/latest/docs/gallery/burtin.html) is cool



Need to look over more of the [biopython documentation](http://biopython.org/DIST/docs/tutorial/Tutorial.html)


...especially the [entrez stuff](http://biopython.org/DIST/docs/api/Bio.Entrez-module.html)

(see also [here](https://www.ncbi.nlm.nih.gov/books/NBK25501/pdf/Bookshelf_NBK25501.pdf))



as an aside, see data viz stuff [here](https://marcobonzanini.com/category/data-visualisation/)

## Setup

Importage

In [11]:

# Generic Imports

import os,sys,glob,numpy as np




# Visualization stuff

%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns


# Analysis stuff

import numpy as np
import pandas as pd

from Bio import Entrez

Define some variables

## Parsing Functions

Define some functions for pulling and parsing data from pubmed

In [102]:
# you need to install Biopython:
# pip install biopython

# Full discussion:
# https://marcobonzanini.wordpress.com/2015/01/12/searching-pubmed-with-python/

def search(query,db='pubmed', sort='relevance', retmax='20',retmode='xml'):
    Entrez.email = 'j.davidgriffiths@gmail.com',
    handle = Entrez.esearch(db=db,sort=sort,retmax=retmax,retmode=retmode,term=query)
    results = Entrez.read(handle)
    return results

def fetch_details(id_list, db='pubmed', retmode='xml'):
    ids = ','.join(id_list)
    Entrez.email = 'j.davidgriffiths@gmail.com',
    handle = Entrez.efetch(db=db,retmode=retmode,id=ids)
    results = Entrez.read(handle)
    return results

In [95]:
def make_papers_df(papers):
    
  # NOTE: had to add in lots of shitty hacks to get this to not error on incomplete records
  # There will be many superior ways of doing this. Needs a little playing around with the 
  # entrez api and the returned results to find best option
  # #
    
  res = []
  for p in papers:

    art = p['MedlineCitation']['Article']
    
    # this is a shitty hack
    if 'AuthorList' in art:
        
    
      tit = art['ArticleTitle']    
    
      """
      # this is a shitty hack
      if 'ForeName' in art['AuthorList'][0]: 
        n1 = art['AuthorList'][0]['ForeName']
      else:
        n1 = '????'
    
      if 'LastName' in art['AuthorList'][0]: 
        n2 = art['AuthorList'][0]['LastName']
      else: 
        n2 = '????'
      """
        
      alist = art['AuthorList']
      astr = ''
      for a_it,a in enumerate(alist):
        if 'ForeName' in a:
          astr+= a['ForeName'] + ' ' + a['LastName'] + ','
        elif 'CollectiveName' in a:
          astr+= a['CollectiveName'] + ','
      astr  = astr[:-1]
        
      dt = art['ArticleDate']
    
      # this is ia shitty hack
      if len(dt)>0: 
        yr = dt[0]['Year']
      else: 
        yr = np.nan
             
      res.append([tit,astr,yr])

    
    df = pd.DataFrame(res) 
    df.columns = ['title', 'author_list', 'date']
    df.columns.names = ['details']
    df.index.names = ['record num']
    
  return df

## Parsing - first attempt

In [103]:
results = search('"Rotman Research Institute"[Affiliation]',retmax=50000)
id_list = results['IdList']
papers = fetch_details(id_list)['PubmedArticle'] # Had to add in this modification

In [104]:
df_papers = make_papers_df(papers)

In [106]:
df_papers.to_csv('entrez_rotman_papers_list.csv',encoding='utf-8')

In [107]:
df_papers.to_pickle('entrez_rotman_papers_list.pkl')#,encoding='utf-8')

---


Styling



In [1]:
def display_css(css_file):
  from IPython.display import HTML,display
  display(HTML(open(css_file, 'r').read()))

In [5]:
display_css('/home/jgriffiths/Code/libraries_of_mine/github/workdocs/masters/styles/CFDPython_css_modified_2.css')